## Парсим json-реплеи в csv

В репе MSC было предложено следующее решение:
```python
import json
from google.protobuf.json_format import Parse
from s2clientprotocol import sc2api_pb2 as sc_pb

with open(REPLAY_INFO_PATH) as f:
    info = json.load(f)
REPLAY_PATH = info['path']
REPLAY_INFO_PROTO = Parse(info['info'], sc_pb.ResponseReplayInfo())
```

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import json
from google.protobuf.json_format import Parse
from s2clientprotocol import sc2api_pb2 as sc_pb
import os
from tqdm import tnrange, tqdm_notebook

REPLAY_INFOS = 'D:\\temp\MSC-master\\MSC-master\\replays_infos'

In [4]:
col=['map_name', 'race_p1', 'apm_p1', 'race_p2', 'apm_p2', 'win_player', 'game_loops', 'game_seconds', 'game_version', 'path']

games_df = pd.DataFrame(columns=col)

In [5]:
REPLAY = '0000e057beefc9b1e9da959ed921b24b9f0a31c63fedb8d94a1db78b58cf92c5.SC2Replay'
REPLAY_INFO_PATH = os.path.join(REPLAY_INFOS, REPLAY)
with open(REPLAY_INFO_PATH) as f:
    info = json.load(f)
replay_info = Parse(info['info'], sc_pb.ResponseReplayInfo())
game = {
        'map_name': replay_info.map_name, 
        'race_p1': replay_info.player_info[0].player_info.race_actual, 
        'apm_p1': replay_info.player_info[0].player_apm, 
        'race_p2': replay_info.player_info[1].player_info.race_actual, 
        'apm_p2': replay_info.player_info[1].player_apm , 
        'win_player': replay_info.player_info[0].player_result.result, 
        'game_loops': replay_info.game_duration_loops, 
        'game_seconds': replay_info.game_duration_seconds, 
        'game_version': replay_info.game_version, 
        'path': info['path']
    }
games_df = games_df.append(game, ignore_index=True)

In [6]:
for i in tqdm_notebook(range(1000), desc='hi'):
    pass

HBox(children=(IntProgress(value=0, description='hi', max=1000), HTML(value='')))

In [7]:
games_list = []
for REPLAY in tqdm_notebook(os.listdir(REPLAY_INFOS), desc="Work"):
    REPLAY_INFO_PATH = os.path.join(REPLAY_INFOS, REPLAY)
    with open(REPLAY_INFO_PATH) as f:
        info = json.load(f)
    replay_info = Parse(info['info'], sc_pb.ResponseReplayInfo())
    game = {
        'map_name': replay_info.map_name, 
        'race_p1': replay_info.player_info[0].player_info.race_actual, 
        'apm_p1': replay_info.player_info[0].player_apm, 
        'race_p2': replay_info.player_info[1].player_info.race_actual, 
        'apm_p2': replay_info.player_info[1].player_apm , 
        'win_player': replay_info.player_info[0].player_result.result, 
        'game_loops': replay_info.game_duration_loops, 
        'game_seconds': replay_info.game_duration_seconds, 
        'game_version': replay_info.game_version, 
        'path': info['path']
    }
    games_list.append(pd.Series(game))

HBox(children=(IntProgress(value=0, description='Work', max=64395), HTML(value='')))

Race:
0. NoRace 
1. Terran
2. Zerg
3. Protoss
4. Random

In [14]:
games_list[1]

apm_p1                                                        195
apm_p2                                                        109
game_loops                                                   7279
game_seconds                                              324.978
game_version                                         3.16.1.55958
map_name                                          Путь на Айур РВ
path            D:\Program Files (x86)\StarCraft II\Replays\3....
race_p1                                                         1
race_p2                                                         1
win_player                                                      1
dtype: object

In [9]:
games_df = pd.concat(games_list, axis=1).transpose()

In [11]:
games_df.to_csv('test_example.csv', encoding='utf-8', sep=';')

In [12]:
games_df

,apm_p1,apm_p2,game_loops,game_seconds,game_version,map_name,path,race_p1,race_p2,win_player
0,386,384,20887,932.52,3.16.1.55958,Меха-депо РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,3,2,2
1,195,109,7279,324.978,3.16.1.55958,Путь на Айур РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,1,1,1
2,55,100,14944,667.189,3.16.1.55958,Каталлена РВ (Void),D:\Program Files (x86)\StarCraft II\Replays\3....,1,2,2
3,138,137,17367,775.367,3.16.1.55958,Путь на Айур РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,2,1,1
4,138,162,12032,537.18,3.16.1.55958,Меха-депо РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,2,2,1
5,0,111,56,2.50017,3.16.1.55958,Одиссея РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,2,2,2
6,134,53,7642,341.185,3.16.1.55958,Незваный гость РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,2,1,1
7,85,104,9021,402.751,3.16.1.55958,Меха-депо РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,3,2,1
8,190,172,13107,585.175,3.16.1.55958,Каталлена РВ (Void),D:\Program Files (x86)\StarCraft II\Replays\3....,1,1,2
9,118,162,10959,489.275,3.16.1.55958,Аколит РВ,D:\Program Files (x86)\StarCraft II\Replays\3....,3,3,1
